<a href="https://colab.research.google.com/github/yangjaehoon/solidityStudy/blob/main/%EB%AF%B8%EB%9D%BC%ED%81%B4%ED%94%8C%EB%A0%88%EC%9D%B4_%ED%81%B4%EB%9F%BD%EB%A9%A4%EB%B2%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Google 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# Step 2: 엑셀 파일을 데이터프레임으로 불러오기
import pandas as pd

# 엑셀 파일 경로 (예: "MyDrive/폴더명/파일명.xlsx")
file_path = '/content/drive/MyDrive/whaleclub_data/whale_club_cp_polygon.xlsx'

# 엑셀 파일을 읽어 데이터프레임으로 변환
df = pd.read_excel(file_path)

# 데이터프레임 출력
df.head()  # 첫 5개 행을 출력하여 확인

#print(df.dtypes)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,rank,name,cp,contribution
0,1,Geishaaa,190,1.9%
1,2,BeeSong,180,1.8%
2,3,Miracle1,180,1.8%
3,4,ALiiii,160,1.6%
4,5,naoj2001,160,1.6%


In [2]:
# 기준이 될 cp 값 입력 받기
user_input = int(input("기준이 될 cp 값을 입력하세요: "))

# 전체 상금 입력 받기
total_prize = float(input("전체 상금을 입력하세요: "))

# 입력된 cp 값 기준으로 필터링된 데이터프레임 생성
filtered_df = df[df['cp'] >= user_input]

# contribution 열 삭제
filtered_df = filtered_df.drop(columns=['contribution'])

# CP 열의 모든 값을 더해서 변수에 저장
total_cp = filtered_df['cp'].sum()

# 각 유저의 상금을 계산하여 새로운 열 'individual_prize' 추가
# 각 유저의 'contribution' 비율에 전체 상금을 곱하여 계산
filtered_df['individual_prize'] = filtered_df['cp'] / total_cp * total_prize

filtered_df['name'] = filtered_df['name'].str.replace(" ", "").str.lower() # 공백 제거 소문자로 변경


# 결과 출력
print(filtered_df.head())  # 첫 5개 행을 확인

filtered_df.info()


기준이 될 cp 값을 입력하세요: 10
전체 상금을 입력하세요: 1000
   rank      name   cp  individual_prize
0     1  geishaaa  190         19.587629
1     2   beesong  180         18.556701
2     3  miracle1  180         18.556701
3     4    aliiii  160         16.494845
4     5  naoj2001  160         16.494845
<class 'pandas.core.frame.DataFrame'>
Index: 143 entries, 0 to 142
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rank              143 non-null    int64  
 1   name              143 non-null    object 
 2   cp                143 non-null    int64  
 3   individual_prize  143 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 5.6+ KB


In [3]:
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import auth
from google.auth import default

# Google Drive 및 Google Sheets API 인증
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 스프레드시트 열기
spreadsheet = gc.open("Whale Club Members Database")  # 파일 이름을 사용하여 Google Sheets 파일 열기
worksheet = spreadsheet.get_worksheet(0)  # 첫 번째 시트를 선택

# Pandas 데이터프레임으로 불러오기
members_df = get_as_dataframe(worksheet)

# 네 번째 열의 값들을 중 중복된 값 추출(폴리곤 닉네임)
duplicates_polygon = members_df.iloc[:, 4].dropna()[members_df.iloc[:, 4].duplicated()]
# base
duplicates_base = members_df.iloc[:, 6].dropna()[members_df.iloc[:, 6].duplicated()]
# opbnb
duplicates_opbnb = members_df.iloc[:, 7].dropna()[members_df.iloc[:, 7].duplicated()]
#arbitrum
duplicates_arbitrum = members_df.iloc[:, 8].dropna()[members_df.iloc[:, 8].duplicated()]

# 중복된 값 출력
print("폴리곤 중복 이름:")
print(duplicates_polygon)
# 중복된 값 출력
print("base 중복 이름:")
print(duplicates_polygon)
# 중복된 값 출력
print("opbnb 중복 이름:")
print(duplicates_polygon)
# 중복된 값 출력
print("arbitrum 중복 이름:")
print(duplicates_polygon)

중복된 값들:
99       Tapzilog
113    Ratujayaaa
124        Diwata
157       Monyet 
163        Shadaw
167      Twelve09
169        kakkoi
174      Junaedi 
175        Lailaa
176      arielbng
194        Jhosua
197      Original
198      Original
200    TIMAHPANAS
202       Scrabby
Name: Polygon Nickname\nIf you don't have it, you don't have to write it down, dtype: object


In [ ]:
# members_df의 2번째 열을 'Name'으로, 11번째 열을 'address'로 선택하여 새로운 데이터프레임 생성
members_df_filtered = members_df.iloc[:, [2,3,4, 11]]
members_df_filtered.columns = ['nickname','discord_id','Name', 'address']  # 열 이름 설정
members_df_filtered['Name'] = members_df_filtered['Name'].str.replace(" ", "").str.lower() # 공백 제거 소문자로 변경
print(filtered_df.head())


# filtered_df의 name 열과 members_df의 Name 열을 기준으로 병합 후 결과를 final_polygon_prize에 저장
final_polygon_prize = filtered_df.merge(
    members_df_filtered,
    left_on='name',  # filtered_df의 이름 열 ('name'으로 가정)
    right_on='Name',  # members_df의 이름 열
    how='left'
)

# 불필요한 'Name' 열 삭제 (optional)
#final_polygon_prize.drop(columns=['Name'], inplace=True)

# final_polygon_prize를 Excel 파일로 저장
final_polygon_prize.to_excel('/content/drive/MyDrive/whaleclub_data/final_polygon_prize.xlsx', index=False)

# 결과 확인
print(final_polygon_prize.head())



Empty DataFrame
Columns: [rank, name, cp, individual_prize]
Index: []


<ipython-input-8-4078a8edc1cc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  members_df_filtered['Name'] = members_df_filtered['Name'].str.replace(" ", "").str.lower() # 공백 제거 소문자로 변경


Empty DataFrame
Columns: [rank, name, cp, individual_prize, nickname, discord_id, Name, address]
Index: []
